<a href="https://colab.research.google.com/github/otome-rin/ensyu-2/blob/main/imitation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install librosa numpy matplotlib scipy fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512549 sha256=dacea3c99017f521ff83f8384c46badb82860ae8b7aaf5a173b8c4941b66e0c7
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [49]:
import librosa
import numpy as np
from fastdtw import fastdtw
from scipy.signal import correlate
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
def extract_f0_dtw(audio_path):
    # 音声データを読み込む
    y, sr = librosa.load(audio_path, sr=None)

    # ピッチを抽出
    pitches, magnitudes = librosa.core.piptrack(y=y, sr=sr)

    # 基本周波数（F0）を取得
    f0 = []
    for t in range(pitches.shape[1]):
        index = magnitudes[:, t].argmax()  # 最大の強度を持つインデックスを取得
        pitch = pitches[index, t]
        if pitch > 0:  # 有効なピッチのみを使用
            f0.append(pitch)

    return f0 if f0 else 0  # 平均基本周波数を返す（データがない場合は0）
# 基本周波数の類似度を計算する関数
def compare_pitch(y1, sr1, y2, sr2):
    # 音声の基本周波数（F0）を抽出
    Max_f0 = 500
    Min_f0 = 100
    f0_1, _, _ = librosa.pyin(y1, fmin = Min_f0, fmax = Max_f0)
    f0_2, _, _ = librosa.pyin(y2, fmin = Min_f0, fmax = Max_f0)

    # NaNを除去
    f0_1 = f0_1[~np.isnan(f0_1)]
    f0_2 = f0_2[~np.isnan(f0_2)]
    # fastdtwを使用して基本周波数の動的時間伸縮距離を計算
    distance, _ = fastdtw(f0_1, f0_2)
    return distance

# 抑揚（ピッチとエネルギー変動）の類似度を計算する関数
def compare_intonation(y1, sr1, y2, sr2):
    # ピッチ（F0）とエネルギー（RMS）の変動を抽出
    rms_1 = librosa.feature.rms(y=y1)[0]
    rms_2 = librosa.feature.rms(y=y2)[0]

    # ピッチとエネルギーの変動を一緒に比較
    #features_1 = np.vstack((f0_1, rms_1)).T
    #features_2 = np.vstack((f0_2, rms_2)).T

    # fastdtwを使用して抑揚の動的時間伸縮距離を計算
    distance, _ = fastdtw(rms_1, rms_2)
    return distance
def sub(audio_path1, audio_path2):
    y1, sr1 = librosa.load(audio_path1)
    y2, sr2 = librosa.load(audio_path2)

    # 基本周波数の類似度
    pitch_similarity = compare_pitch(y1, sr1, y2, sr2)

    # 抑揚の類似度
    intonation_similarity = compare_intonation(y1, sr1, y2, sr2)
    return pitch_similarity,intonation_similarity

def extract_mfcc(file_path, n_mfcc=13):
    # 音声ファイルを読み込み
    y, sr = librosa.load(file_path, sr=None)

    # MFCC特徴量を抽出
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

    # フレームごとの平均値を計算して特徴ベクトルを生成
    mfcc_mean = np.mean(mfcc.T, axis=0)
    return mfcc_mean

# コサイン類似度を計算する関数
def cosine_similarity(vec1, vec2):
    return 1 - cosine(vec1, vec2)

# 話者の類似性を評価する関数
def evaluate_similarity(file1, file2):
    # 2つの音声ファイルからMFCCを抽出
    mfcc1 = extract_mfcc(file1)
    mfcc2 = extract_mfcc(file2)

    # コサイン類似度を計算
    similarity = cosine_similarity(mfcc1, mfcc2)

    return similarity

# 音声データをロードする関数
def load_audio(file_path, sr=22050):
    audio, _ = librosa.load(file_path, sr=sr)
    return audio

# ゼロクロス率を計算する関数
def compute_zero_crossing_rate(audio, frame_length=2048, hop_length=512):
    return librosa.feature.zero_crossing_rate(audio, frame_length=frame_length, hop_length=hop_length)

# テンポ（BPM）を計算する関数
def compute_tempo(audio, sr=22050):
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    return tempo

# ゼロクロス率の類似度を計算する関数
def zero_crossing_rate_similarity(zcr1, zcr2):
    # ゼロクロス率の平均を比較
    zcr1_mean = np.mean(zcr1)
    zcr2_mean = np.mean(zcr2)
    return 1 - np.abs(zcr1_mean - zcr2_mean) / max(zcr1_mean, zcr2_mean)

# テンポの類似度を計算する関数
def tempo_similarity(tempo1, tempo2):
    return 1 - np.abs(tempo1 - tempo2) / max(tempo1, tempo2)


# リズムの類似度を計算
def rhythm_similarity(correlation):
    # 最大相互相関値を取る位置を特定
    max_correlation = np.max(np.abs(correlation))
    # 類似度（最大相互相関値を基準にする）
    similarity = max_correlation
    return similarity
def calculate_cross_correlation(wave1, wave2):
    # 相互相関を計算
    correlation = correlate(wave1, wave2, mode='full')
    return correlation
# メイン関数: 2つの音声ファイルを比較
def compare_audio(file1, file2):
    #音声ファイルをロード
    audio1 = load_audio(file1)
    audio2 = load_audio(file2)
    p,i=sub(file1,file2)
    #  ゼロクロス率を計算
    #zcr1 = compute_zero_crossing_rate(audio1)
    #zcr2 = compute_zero_crossing_rate(audio2
    # 相互相関を計算
    #correlation = correlate(audio1, audio2, mode='full')

    # 3. テンポ（BPM）を計算
  #  tempo1 = compute_tempo(audio1)
  #  tempo2 = compute_tempo(audio2)

    # テンポの類似度を計算
   # tempo_sim = tempo_similarity(tempo1, tempo2)

    # 結果を表示
  # t1=tempo1[0]
  # t2=tempo2[0]
  # ts=tempo_sim[0]
    #print(f"Tempo (BPM) for Audio 1: {t1:.4f}")
    #print(f"Tempo (BPM) for Audio 2: {t2:.4f}")
    #print(f"Tempo Similarity: {ts:.4f}")
    similarity_score00 = evaluate_similarity(file1, file2)
    Max_f0 = 500
    Min_f0 = 100
    f1, _, _ = librosa.pyin(audio1, fmin = Min_f0, fmax = Max_f0)
    f2, _, _ = librosa.pyin(audio2, fmin = Min_f0, fmax = Max_f0)

    # NaNを除去
    f1 = f1[~np.isnan(f1)]
    f2 = f2[~np.isnan(f2)]
    # fastdtwを使用して基本周波数の動的時間伸縮距離を計算
    _,F=fastdtw(f1,f2)
    if(len(F)==0):
      print("False")
      return "False"
     #テンポの類似性
    ry=0
    for i in range(len(F)-1):
      ry+=abs(F[i+1][0]-F[i][0]-F[i+1][1]+F[i][1])
    ry=ry/len(F)
    # 波1と波2の相互相関を計算
    correlation = calculate_cross_correlation(audio1, audio2)
    # リズムの類似度を計算
    similarity = rhythm_similarity(correlation)
    # 結果の表示
    print(file2)
    print(f"Pitch Similarity (F0): {p}")
    print(f"Intonation Similarity (Energy): {i}")
    print(f"Similarity Score: {similarity_score00}")
    print(f"リズムの類似度: {ry}")
    print("")





In [37]:
doraemon1 = 'doraemon.mp3'
doraemon2 = 'doraemon02.mp3'
doraemon3 = 'doraemon03.mp3'
doraemon4 = 'doraemon04.mp3'
doraemon5 = 'doraemon05.mp3'
similarrin1='similar-rin-01.mp3'
similarrin2='similar-rin-02.mp3'
differrin1='differ-rin-01.mp3'
differrin2='differ-rin-02.mp3'
similarfumiki1='similarFumiki1.mp3'
similarfumiki2='similarFumiki2.mp3'
differfumuki1='differFumiki1.mp3'
differfumuki2='differFumiki2.mp3'
effect = 'effect.mp3'
obake = 'obake.mp3'
differ01='differ01.mp3'
differ02='differ02.mp3'
similar01='similar01.mp3'
similar02='similar02.mp3'
similar03='similar03.mp3'
nitenaidora='nitenaidora.mp3'
niterudora='niterudora.mp3'


In [50]:
compare_audio(doraemon1, nitenaidora)

compare_audio(doraemon1, niterudora)

compare_audio(doraemon1, obake)

compare_audio(doraemon1, effect)

compare_audio(doraemon2, similar01)

compare_audio(doraemon2, similar02)

compare_audio(doraemon1, similar03)

compare_audio(doraemon1, similarrin1)

compare_audio(doraemon1, similarrin2)

compare_audio(doraemon1, differrin1)

compare_audio(doraemon1, differrin2)

compare_audio(doraemon2, doraemon1)

compare_audio(doraemon3, doraemon1)

compare_audio(doraemon4, doraemon1)

nitenaidora.mp3
Pitch Similarity (F0): 8888.52264399506
Intonation Similarity (Energy): 93
Similarity Score: 0.6669269222505045
リズムの類似度: 0.7473684210526316

niterudora.mp3
Pitch Similarity (F0): 3910.7835781478775
Intonation Similarity (Energy): 93
Similarity Score: 0.7028380038409291
リズムの類似度: 0.7263157894736842

False
effect.mp3
Pitch Similarity (F0): 7312.779044745884
Intonation Similarity (Energy): 50
Similarity Score: 0.7759510946547562
リズムの類似度: 0.9615384615384616

similar01.mp3
Pitch Similarity (F0): 2287.889495883465
Intonation Similarity (Energy): 87
Similarity Score: 0.9796988741512499
リズムの類似度: 0.5842696629213483

similar02.mp3
Pitch Similarity (F0): 4912.88363453535
Intonation Similarity (Energy): 123
Similarity Score: 0.9222091597934519
リズムの類似度: 0.784

similar03.mp3
Pitch Similarity (F0): 4344.833624886142
Intonation Similarity (Energy): 56
Similarity Score: 0.8223804577949505
リズムの類似度: 0.8448275862068966

similar-rin-01.mp3
Pitch Similarity (F0): 5312.888342221029
Intonation 